# Massive star exploration
This notebook demonstrates the nucleosynthesis exploration of massive star models in the NuGrid data set. This notebook can be executed on the Outreach Hub of the Astrohub https://astrohub.uvic.ca.

This notebook makes the following types of plots/analysis:
* Kippenhahn diagram for given mass and metallicity
* abundance plots for pre-SN and SN models for final or given plot
* movies of abundance evolution for given model range and model step
* integration of abundance taking into account individual remnant masses as a function of initial and final mass

With this notebook you can explore the origin and evolution of any species in massive stars (can be adapted to use for low-mass stars as well of course), specifically radioactive species, such as $\mathrm{^{26}Al}$ and $\mathrm{^{60}Fe}$.

Author: Falk Herwig, 2021.

In [ ]:
%pylab ipympl
from nugridpy import mesa as ms
from nugridpy import nugridse as mp
data_dir="/data/ASDR/NuGrid"  
ms.set_nugrid_path(data_dir)
mp.set_nugrid_path(data_dir)

In [ ]:
# set cycling combination of color-blind labels, glyphs, styles
lll= 2*['-', '--', ':', '-.']
markers = ['X','h','<','>','s','^','d','X','p']
random.shuffle(lll)
CB_color_cycle = ['#4daf4a', '#a65628', '#984ea3',
                  '#ff7f00', '#f781bf', '#377eb8',
                  '#999999', '#e41a1c', '#dede00']
rc('axes', prop_cycle=(cycler('color', CB_color_cycle[0:8]) + cycler('marker',markers[0:8])+cycler('linestyle',lll)))
rc('axes', prop_cycle=(cycler('color', CB_color_cycle[0:8]) +cycler('linestyle',lll)))

# SMALL_SIZE = 8
# MEDIUM_SIZE = 10
# BIGGER_SIZE = 12

# rc('font', size=MEDIUM_SIZE)          # controls default text sizes
# rc('axes', titlesize=SMALL_SIZE)     # fontsize of the axes title
# rc('axes', labelsize=SMALL_SIZE)    # fontsize of the x and y labels
# rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
# rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
# rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
# rc('figure', titlesize=MEDIUM_SIZE)  # fontsize of the figure title


Select intial mass and metallicity:

In [ ]:
mass=20;Z=0.001;ylims=(0,10)

In [ ]:
%%capture --no-display
ifig=127;close(ifig);figure(ifig,figsize=(12,7))
s=ms.star_log(mass=mass,Z=Z)                               # comment for repeat call of this cell with same (M,Z)
mods =(s.get('model_number')[0],s.get('model_number')[-1]) # all models, select more appropriate range:
mods =(4000,8000)
rc('axes', prop_cycle=(cycler('color', CB_color_cycle[0:8]) +cycler('linestyle',lll)))
s.kip_cont(ifig=ifig,modstart=mods[0],modstop=mods[1],ylims=ylims,xres=2000,yres=2000)
title(f'M= {mass:2.0f} Z= {Z:5.4f}'); tight_layout()

Plot the last available cycle from the pre-supernova evolution, or some range as determined from inspecting the Kippenhahn diagram.

In [ ]:
%%capture --no-display
pt=mp.se(mass=mass,Z=Z)  # comment for repeat call of this cell with same (M,Z)
model=int(pt.se.cycles[-1]); model = 4500
species=['H-1','He-4','C-12','C-13','N-14','O-16','Ne-20','Ne-22','Al-26','Si-28','Fe-60']
for mod in range(model,model+0+1,20): # detailed profiles area available every 20 models
    ifig=128;close(ifig);figure(ifig,figsize=(12,7))
    pt.abu_profile(isos=species, ifig=ifig, fname=mod, logy=True,colourblind=False)
    ylim(-7,0);xlim(0,10)
    title(f'M= {mass:2.0f} Z= {Z:5.4f}, model = {mod:4}, pre-SN');tight_layout()
    savefig(f'M{mass:2.0f}Z{Z:5.4f}mod{mod:4}.png')
tight_layout()

Plot explosive nucleosynthesis (use `type='ppd_exp'` option):

In [ ]:
%%capture --no-display
pt=mp.se(mass=mass,Z=Z,type='ppd_exp')  # comment for repeat call of this cell with same (M,Z)
model=int(pt.se.cycles[-1])#; mod = 8000
species=['H-1','He-4','C-12','C-13','N-14','O-16','Ne-20','Ne-22','Al-26','Si-28','Sc-47','Fe-60']
for mod in range(model,model+1,20): # detailed profiles area available every 20 models
    ifig=138;close(ifig);figure(ifig,figsize=(12,7))
    pt.abu_profile(isos=species, ifig=ifig, fname=mod, logy=True,colourblind=False)
    ylim(-7,0);xlim(0,10)
    title(f'M= {mass:2.0f} Z= {Z:5.4f}, model = {mod:4}, SN');tight_layout()
    savefig(f'M{mass:2.0f}Z{Z:5.4f}mod{mod:4}.png')


The profile generating cells are constructed so that if a range is actually given in the `for` loop then a sequence of `.png` files will be produced. Paste the following line into a file called `movies.sh`
```
ffmpeg  -framerate 12   -y -f image2  -pattern_type glob -i "$1*.png" -preset slow -crf 18  -s 1024x1024  -c:v libx264 -b:v 12000k  -pix_fmt yuv420p  $1.mp4
```
Then make this shell script executable with `chmod u+x movies.sh` and make a movie out of your images. Select resulting `.mp4` file in filemanager of JupyterHub and select `Open in New Browser Tab` to play movie.

## Integral of all models
Include remnent masses from Ritter+18, Tab 4.

In [ ]:
from scipy import integrate
def get_isomass_sum(Z,mass,isos,remnant_mass,expl='delay'):
    '''Return radial mass integral for each iso in isos
    '''
    pt=mp.se(mass=mass,Z=Z,type='ppd_exp') 
    model=int(pt.se.cycles[-1])
    isosums = []
    mass =  pt.get(model,'mass')
    rmass = remnant_mass[expl][20][remnant_mass['Z'].index(Z)]
    ind=where(mass>rmass)
    for iso in isos:
        iso = pt.get(model,iso)
        isosums.append(integrate.simps(iso[ind],mass[ind]))
    return isosums

### Remnant masses
According to Ritter+ 2018 (NuGrid Set 1 extension) Tab 4. 

![Ritter+18Tab4-remnantmasses.png](./Ritter+18Tab4-remnantmasses.png)

In [ ]:
remnant_mass = {} # add more masses according to Table
remnant_mass['delay'] = {}; remnant_mass['rapid'] = {}
remnant_mass['Z'] = [0.02, 0.01, 0.006, 0.001, 0.0001]
remnant_mass['delay'][20] = [2.73,2.77,2.79,2.81,2.82]
remnant_mass['rapid'][20] = [2.7,1.83,1.77,1.76,1.76]

### Extract yields for grid and selection of isos

In [ ]:
masses = [12, 15, 20, 25 ]
masses=masses[2:3] # only do 20Msun
Zs = [0.02, 0.01, 0.006, 0.001, 0.0001]

In [ ]:
%%capture
isos = ['Al-26','Fe-60'] # pick isos to ext
data = {}
for expl in ['delay','rapid']:
    data[expl] = {}
    for mass in masses:
        data[expl][mass] = []
        for Z in Zs:
            data[expl][mass].append(get_isomass_sum(Z,mass,isos,remnant_mass,expl=expl))
print(data)    

#### Plot yields as a function of Z

In [ ]:
ifig=536;close(ifig);figure(ifig)
rc('axes', prop_cycle=(cycler('color', CB_color_cycle[0:8]) + cycler('marker',markers[0:8])+cycler('linestyle',lll)))
for expl in ['delay','rapid']:
    for mass in masses:
        for i,iso in enumerate(isos):
            plot(log10(Zs),log10(array(data[expl][mass]).T[i]),label=expl+f' m{mass:2}, '+iso)
legend();xlabel('log Z');ylabel('$\log \int X_i dm$');tight_layout()